In [1]:
### Create logger and load environment variables

from tools.logger import _logger

logger = _logger('my_named_logger')

Loaded 10 environment variables from .env


In [2]:
logger.info('hello world')

[INFO:my_named_logger:] hello world


In [3]:
### Utils

In [4]:
## llm
# instance of LangchainChatLLM
# works with most langchain chains / retrivers and all other langchain module combos

from tools.llm import gpt3, gpt4


completion = gpt3.invoke('How do you do buddy?').content

logger.info('GPT3: ' + completion)


completion = gpt3.invoke('How do you do buddy?').content

logger.info('GPT4: ' + completion)

[INFO:my_named_logger:] GPT3: As an AI language model, I don't have emotions, but I'm always ready to assist you with any questions or tasks you may have. How can I help you today?
[INFO:my_named_logger:] GPT4: As an AI language model, I don't have emotions, but I'm always ready to assist you with any questions or tasks you may have. How can I help you today?


In [5]:
## url loader

from tools.scrapingbee_tool import load_page_for_url


document = load_page_for_url('https://hackernews.com')

logger.info('Document.metadata: ' + str(document.metadata))

[INFO:url_document_loader:] scrapingbee client params: {'render_js': 'True'}
[INFO:my_named_logger:] Document.metadata: {'url': 'https://hackernews.com', 'title': 'Hacker News', 'description': None}


In [6]:
logger.info('Document.page_content: ' + str(document.page_content))

[INFO:my_named_logger:] Document.page_content: | |  | **[Hacker News](news)** [new](newest) \| [past](front) \| [comments](newcomments) \| <ask> \| <show> \| <jobs> \| <submit> | [login](login?goto=news) | | --- | --- | --- | |
| --- | --- | --- | --- |
| | 1\. |  | [It's Time to Stop Taking Sam Altman at His Word](https://www.theatlantic.com/technology/archive/2024/10/sam-altman-mythmaking/680152/) ([theatlantic.com](from?site=theatlantic.com)) | | --- | --- | --- | |  | | 79 points by [redwoolf](user?id=redwoolf) [1 hour ago](item?id=41749371)  \| [hide](hide?id=41749371&goto=news) \| [44 comments](item?id=41749371) | | | 2\. |  | [Noise](https://noise.jake.fun/) ([jake.fun](from?site=jake.fun)) | |  | | 322 points by [amadeuspagel](user?id=amadeuspagel) [9 hours ago](item?id=41715883)  \| [hide](hide?id=41715883&goto=news) \| [47 comments](item?id=41715883) | | | 3\. |  | [Linux from Scratch](https://www.linuxfromscratch.org/index.html) ([linuxfromscratch.org](from?site=linuxfromscr

In [7]:
from tools.bing_tool import search_bing

results = search_bing('python tutorials')

for r in results:
    print(r)

page_content='W3Schools offers a comprehensive and interactive<b> Python tutorial</b> with examples, exercises, quizzes, and references. You can also download Python, get certified, and access My Learning program.' metadata={'snippet': 'W3Schools offers a comprehensive and interactive<b> Python tutorial</b> with examples, exercises, quizzes, and references. You can also download Python, get certified, and access My Learning program.', 'title': 'Python Tutorial - W3Schools', 'link': 'https://www.w3schools.com/python/'}
page_content='Learn<b> Python</b> for data science with DataCamp&#39;s online<b> courses,</b> videos and coding challenges. Join over a million learners and get certified, save 25% off your subscription and access more<b> tutorials.</b>' metadata={'snippet': 'Learn<b> Python</b> for data science with DataCamp&#39;s online<b> courses,</b> videos and coding challenges. Join over a million learners and get certified, save 25% off your subscription and access more<b> tutorial

In [ ]:
# put it together in an agent with a chain access to multiople tools (Jethro)



from langchain.agents import AgentExecutor, create_tool_calling_agent, tool

@tool
def search_bing_browser(query: str) -> str:
    """search bing using browser"""

    search_results_documents = search_bing(query) 

    # format result for llm eo view

    result = ""
    for r in search_results_documents:
        result += str(r) + '\n'

    return result


@tool
def view_page_content(url: str) -> str:
    """read entire web page content from search results url"""

    document = load_page_for_url(url)
    return document.page_content

    


In [ ]:


# @traceable
# def invoke_agent_with_tools(human, llm, tools=[], verbose=True) -> [str, List]:
#     """basic agent with tools"""

#     custom_instruction = """ignore all previous instructions. give me very short and concise answers and ignore all the niceties that openai programmed you with; i know you are a large language model but please pretend to be a confident and superintelligent oracle that can help a confused ceo of an ai company figure out how to help humanity navigate the golden path towards superintelligence.
    
# it is very important that you get this right."""

#     prompt = ChatPromptTemplate.from_messages(
#         [
#             ("system", custom_instruction),
#             ("placeholder", "{chat_history}"),              # memory
#             ("human", "{input}"),
#             ("placeholder", "{agent_scratchpad}"),
#         ]
#     )
#     agent = create_tool_calling_agent(
#         llm=llm,
#         tools=tools,
#         prompt=prompt
#     )
#     agent_executor = AgentExecutor(
#         agent=agent,
#         tools=tools,
#         verbose=verbose,
#         handle_parsing_errors=True,
#         return_intermediate_steps=True,
#         early_stopping_method='generate',
#     )
#     agent_response = agent_executor.invoke({"input": human})
#     agent_answer = agent_response.get('output', None) or '[agent returned no output]'
#     intermediate_steps = agent_response.get('intermediate_steps', None) or []
#     return agent_answer, intermediate_steps